In [1]:
import json
import glob
import numpy as np
from src.evaluation import StrategyQA_Evaluator

2024-06-14 15:12:37.477396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 15:12:37.477459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 15:12:37.479345: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 15:12:37.487315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 15:12:38.686669: W tensorflow/compiler/tf2

In [3]:
strqa = StrategyQA_Evaluator(split='test')

In [4]:
with open("data/strategyqa/strategyqa_dev_idx.json") as f:
    dev_idx = json.load(f)

with open('data/strategyqa/strategyqa_test_idx.json') as f:
    test_idx = json.load(f)

In [72]:
base_path = "models/models--llama-2-hf/13B/DCoT/seed_42/checkpoint-24024/test/temp_0.0/StrategyQA/"

In [73]:
files = glob.glob(base_path+"*")
for f in files:
    if "results.json" not in f:
        with open(f) as fp:
            outputs = json.load(fp)
        test_outputs = np.array(outputs)[test_idx]
        with open(f, "w") as fp:
            json.dump(test_outputs.tolist(), fp, indent=4)

In [74]:
with open(base_path + "list_preds.json") as f:
    preds = json.load(f)
results = strqa.evaluate(test_outputs.tolist())
print(results)
with open(base_path + "list_preds.json", "w") as f:
    json.dump(results, f)

{'A': {'precision': 0.7631578947368421, 'recall': 0.4915254237288136, 'f1-score': 0.597938144329897, 'support': 177.0}, 'B': {'precision': 0.6086956521739131, 'recall': 0.8383233532934131, 'f1-score': 0.7052896725440807, 'support': 167.0}, 'accuracy': 0.6598837209302325, 'macro avg': {'precision': 0.6859267734553776, 'recall': 0.6649243885111134, 'f1-score': 0.6516139084369889, 'support': 344.0}, 'weighted avg': {'precision': 0.6881718641903039, 'recall': 0.6598837209302325, 'f1-score': 0.6500535664571315, 'support': 344.0}}


In [75]:
results

{'A': {'precision': 0.7631578947368421,
  'recall': 0.4915254237288136,
  'f1-score': 0.597938144329897,
  'support': 177.0},
 'B': {'precision': 0.6086956521739131,
  'recall': 0.8383233532934131,
  'f1-score': 0.7052896725440807,
  'support': 167.0},
 'accuracy': 0.6598837209302325,
 'macro avg': {'precision': 0.6859267734553776,
  'recall': 0.6649243885111134,
  'f1-score': 0.6516139084369889,
  'support': 344.0},
 'weighted avg': {'precision': 0.6881718641903039,
  'recall': 0.6598837209302325,
  'f1-score': 0.6500535664571315,
  'support': 344.0}}

In [76]:
base_path = "models/models--llama-2-hf/13B/DCoT/seed_42/checkpoint-24024/test/temp_0.7/self_consistency/StrategyQA/"

In [77]:
files = glob.glob(base_path+"*")
for f in files:
    if "results.json" not in f:
        with open(f) as fp:
            outputs = json.load(fp)
        test_outputs = np.array(outputs)[test_idx]
        with open(f, "w") as fp:
            json.dump(test_outputs.tolist(), fp, indent=4)

In [78]:
with open(base_path + "list_preds.json") as f:
    test_outputs = json.load(f)
results = dict()
for k in range(1, 6):
    list_preds_k = []
    for preds in test_outputs:
        list_preds_k.append(strqa._most_common(preds[:k]))
    results_k = strqa.evaluate(list_preds_k)
    results[k] = results_k

In [79]:
for i, vals in results.items():
    print(i, vals['macro avg']['f1-score'])

1 0.642349679431641
2 0.642349679431641
3 0.6409560588665066
4 0.6454599447996366
5 0.6464728978365701


In [80]:
with open(base_path + "list_preds.json", "w") as f:
    json.dump(results, f)